##IMPORT THƯ VIỆN VÀ LOAD DATA

In [1]:
!pip install underthesea
!pip install openpyxl ##engine
!pip install gensim==3.4.0
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import class_weight as cw
from sklearn import utils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from underthesea import word_tokenize
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import multiprocessing
# Word2vec
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
df = pd.read_csv('/content/drive/MyDrive/clab/full.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 1.3 MB/s 
     |████████████████████████████████| 965 kB 58.3 MB/s 
     |████████████████████████████████| 235 kB 61.8 MB/s 
     |████████████████████████████████| 581 kB 33.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.2 MB 53.6 MB/s 
  Created wheel for gensim: filename=gensim-3.4.0-cp37-cp37m-linux_x86_64.whl size=23316742 sha256=327aa5d23ac5d54ad7f51b2f350c8ed445c96a1f55dcde28c117f83f330d9e20
  Stored in directory: /root/.cache/pip/wheels/de/a4/46/4e18f7d25915b16e0e790a5362e455aba6cadc486994806c05
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Su

##TIỀN XỬ LÝ DỮ LIỆU

In [26]:
def remove_k_dau(df):
    alpha_vn ="ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđẠẢÃÀÁÂẬẦẤẨẪĂẮẰẶẲẴÓÒỌÕỎÔỘỔỖỒỐƠỜỚỢỞỠÉÈẺẸẼÊẾỀỆỂỄÚÙỤỦŨƯỰỮỬỪỨÍÌỊỈĨÝỲỶỴỸĐ"
    index=[]
    for i,text in enumerate(df.comment.astype(str)):
        words = text.split(' ')
        k_dau = []
        if len(words) >=5:
            for word in words:
                for char in alpha_vn:
                    if char in word:
                        k_dau.append(word)  
            if len(k_dau)==0:
                index.append(i)    
    df=df.drop(index,axis=0)
    return df

dic = pd.read_excel('/content/dictionary_new.xlsx','Sheet1')
fullcom = dic.set_index('Short').T.to_dict('list')# biến đổi câu có từ viết tắt thành câu hoàn chỉnh
def slang(text) : 
    result = []
    for word in text.split() : 
        try :
            word = fullcom[word][0]
            result.append(word)
        except :
            result.append(word)

    sentence=' '.join(word for word in result)
    return sentence
# tạo stopword
with open('/content/vietnamese_stopword_2.txt') as f:
    data = f.read()
new_stopword = [ ]
stopword_list = [line for line in data.split('\n') if line != '']
stopword_list=stopword_list + new_stopword
stop_word= set(stopword_list)
stop_words=[]
for word in stop_word:
    stop_words.append(word.replace(" ","_"))
# tạo elongated: vuiiii -> vui
import re
from itertools import groupby

# Xóa emoji
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
#Loại bỏ spam : 
def spam(text) :
    sentence = []
    for word in text.split() :
        if len(word) < 6 :
           sentence.append(word) 
        else :
            word = " "    
    sentence=' '.join(word for word in sentence)
    return sentence

# biến đổi câu có từ viết tắt thành câu hoàn chỉnh
def slang(text) : 
    result = []
    for word in text.split() : 
        try :
            word = fullcom[word][0]
            result.append(word)
        except :
            result.append(word)

    sentence=' '.join(word for word in result)
    return sentence
# dùng thư viện underthesea để nối từ
def convert_word_tokenize(text):
    return word_tokenize(text,format='text')

def elongated_remove(text):
    return ''.join(c for c, _ in groupby(text))

# tạo negation: ví dụ câu: Tôi không thích đi học -> Tôi không_thích đi học
def negation_handling(text):
    negation = False
    result = []
    words = text.split()
    for word in words:
        stripped = word.strip().lower()
        negated = "không_" + stripped if negation else stripped
        if negation == True:
            negation = not negation
        result.append(negated)
        if any(neg == word for neg in ["không","chưa","chẳng","chả","đừng"
                                       "Không","Chưa","Chả","Chẳng","Đừng"]):
            negation = not negation
    for word in result:
        if word in ["không","chưa","chẳng","chả","đừng"
                    "Không","Chưa","Chả","Chẳng","Đừng"]:
            result.remove(word)
    result = ' '.join(result)
    return result

# tạo intensification: ví dụ câu: Tôi rất thích đi học -> Tôi rất_thích đi học
def intensification_handling(text):
    negation = False
    result = []
    words = text.split()
    for word in words:
        stripped = word.strip().lower()
        negated = "rất_" + stripped if negation else stripped
        if negation == True:
            negation = not negation
        result.append(negated)
        if any(neg == word for neg in ["rất","khá","hơi","quá","toàn",
                                      "Rất","Khá","Hơi","Quá","Toàn"]):
            negation = not negation
    for word in result:
        if word in ['rất','hơi','khá','quá','toàn',
                   "Rất","Khá","Hơi","Quá","Toàn"]:
            result.remove(word)
    result = ' '.join(result)
    return result
  # Xóa code html
def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

# xóa link
def remove_url(document):
    document = re.sub(r"(http\S+)|(www\S+)", '', document)
    return document

# xóa tag user
def remove_tag_user(document):
    document = re.sub(r"(@\w{1,15})", '', document)
    return document

# xóa hashtags
def remove_hashtags(document):
    document = re.sub("#(\w{1,})", '', document)
    return document

# Chuẩn hóa unicode
import regex as re
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# Chuẩn hóa dấu tiếng việt
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']
nguyen_am_to_ids = {}
for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9: 
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5: 
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    return ''.join(chars)
def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True
def chuan_hoa_dau_cau_tieng_viet(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# Chuyển 100k thành giá tiền
def money(text):
    text = re.sub('(\d+k)|(\d+ k)|(\d+đồng)|(\d+ đồng)|(\d+ngàn)|(\d+ ngàn)|(\d+nghìn)|(\d+ nghìn)', 'giá_tiền', text)
    return text# Xóa code html

def remove_stopword(text):
    sentence=[]
    for word in text.split():
        if word not in stop_words:
            sentence.append(word)
    sentence= ' '.join(word for word in sentence)
    return sentence
# xóa số trong câu
def clean_number_object(string: str, punctuations=r'''!+()-[]{};:'"\,<>./?@#$%^&~1234657890''') -> str:
    # Xóa số
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
    return string


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [27]:
def pre_process(data_comment):
    new_comment=[]
    for text in data_comment:
        text = remove_emojis(text)
        text = remove_html(text)
        text = remove_url(text)
        text = remove_tag_user(text)
        text = remove_hashtags(text)
        text = elongated_remove(text)
        text = chuan_hoa_dau_cau_tieng_viet(text)
        text = spam(text)
        text = money(text)
        text = slang(text)
        text = convert_unicode(text)
        text = clean_number_object(text)
        text = intensification_handling(text)
        text = negation_handling(text)
        text = convert_word_tokenize(text)
        text = text.lower()
        text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        text = remove_stopword(text)
        new_comment.append(text)
    return new_comment

In [28]:
df1 = df.drop_duplicates(subset='Description_Prod')
df1 = df1.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Description_Product', 'Index'])
df1 = df1.dropna(subset=["User_id", "User_Rating", "Prod_id"])
df1 = df1.reset_index()
df1 = df1.drop(columns='index')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Shop_id           598 non-null    int64  
 1   Shop_Name         598 non-null    object 
 2   Prod_id           598 non-null    int64  
 3   Prod_Name         598 non-null    object 
 4   Description_Prod  598 non-null    object 
 5   Price             598 non-null    float64
 6   Average_Rating    598 non-null    float64
 7   Total_Comments    598 non-null    int64  
 8   User_Rating       598 non-null    float64
 9   User_id           598 non-null    float64
 10  User_Review       595 non-null    object 
 11  URL               598 non-null    object 
dtypes: float64(4), int64(3), object(5)
memory usage: 56.2+ KB


In [29]:
df1['Description_Prod'] = pre_process(df1['Description_Prod'].astype(str))

##TFIDF

In [30]:
def get_name_food_(index):
    return df1['Prod_Name'].iloc[index]

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
def recommendation_system_food_using_TFTDF_(food_id, number):
  # food index mapping
  mapping1 = pd.Series(df1.index, index = df1['Prod_Name'])
  id1 = mapping1[food_id]
  #print('Id1:',id1)
  #print('List mapping of product name:\n',mapping)
  product1 = mapping1.index[id1]
  #print(product1)

  demo_id1 = mapping1.get(key = product1)
  #print('Demo_id1:',demo_id1)
  mapping2 = pd.Series(df1.index, index = df1['Prod_Name'])
  id2 = mapping2.get(key = product1)
  #print('Id2:',id2)
  product2 = mapping2.index[id2]
  #print(product2)


  # TF-TDF
  vectorizer = TfidfVectorizer()
  tfidf_vectorizer = vectorizer.fit(df1['Description_Prod'])
  #print(tfidf_vectorizer)

  overview_matrix = tfidf_vectorizer.transform(df1['Description_Prod'])
  #print(overview_matrix)
  #print('Overview_matrix shape:',overview_matrix.shape)

  similarity_matrix = linear_kernel(overview_matrix, overview_matrix)
  #print('Similarity_matrix:\n',similarity_matrix)
  #print('Similarity_matrix shape:',similarity_matrix.shape)

  # Recommendation
  print('Product name users need to recommend to them:\n', product2)
  similarity_score = list(enumerate(similarity_matrix[id2]))
  global foodname
  foodname = product2

  similarity_score = sorted(similarity_score, key = lambda x: x[1], reverse = True)
  similarity_score = similarity_score[1:21]
  recommendation = [(get_name_food_(id) , score) for id, score in similarity_score]

  # Xuất ra danh sách top 10 sản phẩm được khuyến nghị cho người dùng với một sản phẩm chỉ định dưới dạng bảng
  recommendation_df = pd.DataFrame(recommendation, columns=['Prod_Name','Cosine_similarity'])
  recommendation_df = pd.merge(df1, recommendation_df, on="Prod_Name", how="inner")
  recommendation_df = recommendation_df.sort_values(by=['Cosine_similarity'],ascending=False)
  recommendation_df = recommendation_df.head(number)
  print(recommendation_df['Prod_Name'])

  return recommendation_df

In [32]:
recommendation_system_food_using_TFTDF_(200, 5)

Product name users need to recommend to them:
 Hủ 200gr ruốc xấy ngon số 1 TÂY NINH dùng chung bánh 
18                           500g Ruốc Sấy Tây Ninh Giò
13    {Ruốc gà thịt tươi 💯}Hạnh Gà/ Ruốc Gà - chà bô...
16                 Đồ Ăn Nhanh Ruốc Cá Chép,Chà Bông Cá
19          100gr CHÀ BÔNG TÔM/ RUỐC TÔM ĐÀ NẴNG - siêu
11                     Hủ 300G Ruốc Sấy Hành Phi Trọng 
Name: Prod_Name, dtype: object


,Shop_id,Shop_Name,Prod_id,Prod_Name,Description_Prod,Price,Average_Rating,Total_Comments,User_Rating,User_id,User_Review,URL,Cosine_similarity
18,237204869,dacsanphanrangso1,9207719667,500g Ruốc Sấy Tây Ninh Giò,ruốc sấy khô liệu ruốc biển tươi sạch sấy khô ...,59420.60320,2.311383,29,5.0,64154689.0,Video hình ảnh mang tính chất nhận xu ạ. \nSho...,https://shopee.vn/500g-Ru%E1%BB%91c-S%E1%BA%A5...,0.465455
13,50421059,Hạnh Gà,2767491558,{Ruốc gà thịt tươi 💯}Hạnh Gà/ Ruốc Gà - chà bô...,ruốc gà giã bằng gà tươi nên sợi ruốc ăn thơm ...,160283.04651,2.087949,26,5.0,38479513.0,"Ruốc ngon lắm, ai ăn nhạt thì vừa còn ai ai ăn...",https://shopee.vn/-Ru%E1%BB%91c-g%C3%A0-th%E1%...,0.427757
16,667449329,THẢO MỘC ĐỒNG XANH,17103008545,"Đồ Ăn Nhanh Ruốc Cá Chép,Chà Bông Cá",thông_tin sản_phẩm ruốc cá chép ruốc hợp_tác_x...,327683.30011,2.739733,2,5.0,471091541.0,"Giao hàng nhanh, ruốc thơm, ngon. Nhưng thi th...",https://shopee.vn/%C4%90%E1%BB%93-%C4%82n-Nhan...,0.397616
19,63406509,Đặc Sản Đà Nẵng Chính Gốc,12557561629,100gr CHÀ BÔNG TÔM/ RUỐC TÔM ĐÀ NẴNG - siêu,tôm ruốc tôm đà nẵng siêu ngon đặc_điểm sợi ru...,71966.25222,5.998826,2,5.0,257376075.0,Đậm vị quá ngon chất lượng giá hợp lý ship hàn...,https://shopee.vn/100gr-CH%C3%80-B%C3%94NG-T%C...,0.367723
11,17447969,BÁNH TRÁNG TRỌNG NGHĨA,247272852,Hủ 300G Ruốc Sấy Hành Phi Trọng,ruốc sấy hành phi thành_phần ruốc hành phi_hàn...,95462.18337,4.902445,21,4.0,44504253.0,Mình ăn vào lại thấy có vị nhẫn ko biết nguyên...,https://shopee.vn/H%E1%BB%A7-300G-Ru%E1%BB%91c...,0.329758


##WORD2VEC

In [15]:
cores=multiprocessing.cpu_count()

# model dùng Bag of word
SEED = 455 # random
W2V_SIZE = 400
W2V_WINDOW = 9 # max distance bt the current and predicted
W2V_EPOCH = 35 
W2V_MIN_COUNT = 6 # Ignore word frequence < n

w2v_model = Word2Vec(   seed=SEED, 
                        size=W2V_SIZE,
                        window=W2V_WINDOW,
                        min_count=W2V_MIN_COUNT,
                        workers=cores
                        )

In [16]:
documents = [_text.split() for _text in df1['Description_Prod'].astype(str)]

In [17]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
w2v_model.build_vocab(tqdm(documents))

100%|██████████| 598/598 [00:00<00:00, 34281.80it/s]


In [18]:
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

(1787283, 2356165)

In [19]:
w2v_model.most_similar('ngon',topn=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('dai', 0.7435066103935242),
 ('bạch_tuộc', 0.6610574722290039),
 ('thơm', 0.6519355177879333),
 ('đậm_đà', 0.6159470081329346),
 ('hương_vị', 0.6060012578964233),
 ('hấp_dẫn', 0.6024566292762756),
 ('đặc_trưng', 0.5924603939056396),
 ('râu', 0.5866954326629639),
 ('không_lẫn', 0.5854678153991699),
 ('mùi', 0.5647065043449402)]

In [20]:
w2v_model.init_sims(replace=True)
print(w2v_model)

Word2Vec(vocab=1599, size=400, alpha=0.025)


In [21]:
def vectors(df1):
    
    # Creating a list for storing the vectors ('Description' into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each 'Description'
    for line in df1['Description_Prod']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in w2v_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = w2v_model[word]
                else:
                    avgword2vec = avgword2vec + w2v_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)

In [33]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# Recommending the Top 10 similar food
def recommendation_system_food_using_W2v_(prod_id, number):

    # food index mapping
    mapping1 = pd.Series(df1.index, index = df1['Prod_Name'])
    id1 = mapping1[prod_id]
    #print('List mapping of product name:\n',mapping)
    product1 = mapping1.index[id1]

    demo_id1 = mapping1.get(key = product1)
    mapping2 = pd.Series(df1.index, index = df1['Prod_Name'])
    id2 = mapping2.get(key = product1)
    product2 = mapping2.index[id2]
  
    global foodname
    foodname = product2
    # Calling the function vectors
    vectors(df1)
    # Finding cosine similarity for the vectors
    similarity_matrix = cosine_similarity(word_embeddings, word_embeddings)
    #similarity_matrix = linear_kernel(word_embeddings, word_embeddings)


    # Recommendation
    print('Product name users need to recommend to them:\n', product2)
    similarity_score = list(enumerate(similarity_matrix[id2]))
    similarity_score = sorted(similarity_score, key = lambda x: x[1], reverse = True)
    similarity_score = similarity_score[1:21]
    recommendation = [(get_name_food_(id) , score) for id, score in similarity_score]
    # Xuất ra danh sách top 10 sản phẩm được khuyến nghị cho người dùng với một sản phẩm chỉ định dưới dạng bảng
    recommendation_df = pd.DataFrame(recommendation, columns=['Prod_Name','Cosine_similarity'])
    recommendation_df = pd.merge(df1, recommendation_df, on="Prod_Name", how="inner")
    recommendation_df = recommendation_df.sort_values(by=['Cosine_similarity'],ascending=False)
    recommendation_df = recommendation_df.head(number)
    print(recommendation_df['Prod_Name'])
    return recommendation_df


In [34]:
recommendation_system_food_using_W2v_(200, 5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Product name users need to recommend to them:
 Hủ 200gr ruốc xấy ngon số 1 TÂY NINH dùng chung bánh 
13                           500g Ruốc Sấy Tây Ninh Giò
16                                   Quả dâu tây sấy dẻ
8                  100GR Bò khô xé sợi thơm ngon hàng l
18          100gr CHÀ BÔNG TÔM/ RUỐC TÔM ĐÀ NẴNG - siêu
11    [HÀ NỘI] Xúc Xích Sụn Gà Cay Ăn Liền 1 gói 4 v...
Name: Prod_Name, dtype: object


,Shop_id,Shop_Name,Prod_id,Prod_Name,Description_Prod,Price,Average_Rating,Total_Comments,User_Rating,User_id,User_Review,URL,Cosine_similarity
13,237204869,dacsanphanrangso1,9207719667,500g Ruốc Sấy Tây Ninh Giò,ruốc sấy khô liệu ruốc biển tươi sạch sấy khô ...,59420.60320,2.311383,29,5.0,64154689.0,Video hình ảnh mang tính chất nhận xu ạ. \nSho...,https://shopee.vn/500g-Ru%E1%BB%91c-S%E1%BA%A5...,0.895438
16,42004863,huyenpi.kul,10477233552,Quả dâu tây sấy dẻ,dâu_tây sấy dẻo đặc_sản cao_cấp dai dai mềm hư...,29825.88724,2.301471,20,5.0,377287160.0,"Jsjsnsjdjkd hình ảnh minh họa ,mang tc nhận xu...",https://shopee.vn/Qu%E1%BA%A3-d%C3%A2u-t%C3%A2...,0.838509
8,43086726,TLFood Shop,5806774572,100GR Bò khô xé sợi thơm ngon hàng l,gr bò khô sợi heo giả bò món nhậu lý_tưởng đấn...,15134.89551,1.939551,2675,5.0,772005828.0,Hương vị:nzndjdjdkd như cục đất\n\nĐnndnkkwlwk...,https://shopee.vn/100GR-B%C3%B2-kh%C3%B4-x%C3%...,0.826895
18,63406509,Đặc Sản Đà Nẵng Chính Gốc,12557561629,100gr CHÀ BÔNG TÔM/ RUỐC TÔM ĐÀ NẴNG - siêu,tôm ruốc tôm đà nẵng siêu ngon đặc_điểm sợi ru...,71966.25222,5.998826,2,5.0,257376075.0,Đậm vị quá ngon chất lượng giá hợp lý ship hàn...,https://shopee.vn/100gr-CH%C3%80-B%C3%94NG-T%C...,0.823328
11,2770707,Demacia Station - 39 Thụy Khuê,9539017473,[HÀ NỘI] Xúc Xích Sụn Gà Cay Ăn Liền 1 gói 4 v...,xúc_xích sụn non mới ngon lạ_miệng dai dai cắn...,9262.24843,0.588511,2,5.0,404031556.0,Sản phảmmmmm\nM\nM\nMm\nMm\nMok\nOkkkkkkkkkkkk...,https://shopee.vn/-H%C3%80-N%E1%BB%98I-X%C3%BA...,0.819496
